# M4101, Expérimentations autour de Tic-Tac-Toe

Ce calepin est (fortement) inspiré par différents travaux :
- La classe State est inspirée par le code de Jeff Bradberry (https://github.com/jbradberry). 
- Le code des algorithmes d'apprentissage par renforcement est inspiré par le code de  Nathanael Fijalkow (https://github.com/nathanael-fijalkow/TicTacToe_RL)
- MinMax et AlphaBeta sont inspirés de https://stackabuse.com/minimax-and-alpha-beta-pruning-in-python/

## Implémentations de certains algorithmes pour le jeu Tic Tac Toe.

### Optimisation :
* Algorithme Min-Max
* Variante de Min-Max : Alpha-Beta
### Apprentissage par renforcement :
* Algorithmes d'apprentissage par renforcement
  * Moyenne
  * Step size

### Partie 1 - Optimisation : Min-Max et sa variante Alpha-Beta
Nous nous intéressons aux jeux symétriques à somme nulle à deux joueurs comme par exemple : jeu d'échec, Go, Tic-Tac-Toe, Backgammon, Reversi , Dames, Awalee, etc. Pour des raisons de simplicités, nous choisirons comme jeu le Tic-Tac-Toe.
Cette partie est dédiée à l'expérimentation d'un algorithme d'optimisation appelé "Min-Max" sur ce jeu : l'algorithme Min-Max permet une prise de décision optimale en théorie des jeux. Pour être plus efficace, ses performances seront améliorées en utilisant l'élagage alpha-bêta : cette version améliorée sera aussi présentée.

In [1]:
import numpy as np


#### Classe State : 
- utilisée pour gérer les configurations du jeu.

In [2]:
SIZE = 3

class State(object):
    def __init__(self):
        # @data is a SIZE * SIZE array where
        # 0 represents an empty position
        # 1 represents a cross (symbol for player 1)
        # 2 represents a circle (symbol for player 2)
        self.data = np.zeros((SIZE, SIZE))

        # @player: who's turn it is to play from this state
        self.player = 1

        # @outcome can be
        # 1 if Player 1 wins
        # 0 if Player 2 wins
        # 0.5 if it's a tie
        # -1 if the game is not over
        # 2 if the outcome has never been computed
        self.outcome = 2

    # Checks whether the game is over from this state and who won
    def compute_outcome(self):
        if self.outcome != 2:
            return self.outcome
        else:
            # Checks rows
            for i in range(0, SIZE):
                if all(x == 1 for x in self.data[i, :]):
                    self.outcome = 1
                    return 1
                if all(x == 2 for x in self.data[i, :]):
                    self.outcome = 0
                    return 0

            # Checks columns
            for j in range(0, SIZE):
                if all(x == 1 for x in self.data[:, j]):
                    self.outcome = 1
                    return 1
                if all(x == 2 for x in self.data[:, j]):
                    self.outcome = 0
                    return 0

            # Checks diagonals
            diag = [self.data[i,i] for i in range(0, SIZE)]
            if all(x == 1 for x in diag):
                self.outcome = 1
                return 1
            if all(x == 2 for x in diag):
                self.outcome = 0
                return 0

            anti_diag = [self.data[i,SIZE - 1 - i] for i in range(0, SIZE)]
            if all(x == 1 for x in anti_diag):
                self.outcome = 1
                return 1
            if all(x == 2 for x in anti_diag):
                self.outcome = 0
                return 0

            # Checks whether it's a tie
            data_all = [self.data[i,j] for i in range(0, SIZE) for j in range(0, SIZE)]
            if all(x != 0 for x in data_all):
                self.outcome = 0.5
                return 0.5

            # If we reached this point the game is still going on
            self.outcome = -1
            return -1

    # Prints the board
    def print_state(self):
        for i in range(0, SIZE):
            print('-------------')
            out = '| '
            for j in range(0, SIZE):
                if self.data[i, j] == 1:
                    token = 'x'
                elif self.data[i, j] == 2:
                    token = 'o'
                else:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

    # Takes a state and returns the full list of moves that are legal moves
    def legal_plays(self):
        legal = []
        for i in range(0, SIZE):
            for j in range(0, SIZE):
                if self.data[i, j] == 0:
                    legal.append((i,j))
        return legal

    # Returns a new state obtained by playing in (i,j)
    def next_state(self, i, j):
        new_state = State()
        new_state.data = np.copy(self.data)
        new_state.data[i, j] = self.player
        new_state.player = 3 - self.player
        return new_state

    # Returns a new state obtained by copying the current state
    def copy_state(self):
        new_state = State()
        new_state.data = np.copy(self.data)
        new_state.player = self.player
        return new_state

    # Updates the same state by playing in (i,j)
    def update_state(self, i, j):
        self.data[i, j] = self.player
        self.player = 3 - self.player
        self.outcome = 2

### Quelques manipulations élémentaires pour commencer :

- Créer une instance de la classe State et afficher l'état du jeu au démarrage

- Jouer une partie où le joueur 1 gagne


In [3]:
print(' ')

### Quelques manipulations élémentaires (suite) :
- Jouer une partie au hasard (les 2 joueurs jouent au hasard)


In [4]:
import random


C'est la classe Player qui va implementer les differents algorithmes :

* MinMax  
* AlphaBeta
* Random
* Human

Les versions "Human" et "Random" sont déjà écrites : 

In [5]:
class Player():
    def __init__(self, name = 'Anonymous', strategy ='human'):
        self.name = name
        self.strategy = strategy

    def play(self, state, verbose = False):      
        if self.strategy == 'random':  
            if (verbose):
                print("Joueur ",state.player,": Je joue au hasard.")
            return random.choice(state.legal_plays())
        elif self.strategy == 'human':
            if (verbose):
                print("Joueur ",state.player,": Un humain joue...")
            while True:   
                px = int(input('Insert the X coordinate: '))
                py = int(input('Insert the Y coordinate: '))
                if (px, py) in state.legal_plays():
                    return (px, py)
                else:
                    print('The move is not valid! Try again.')
        else:
            print("Je ne connais pas cette stratégie : je joue au hasard !")
            return random.choice(state.legal_plays())

La classe Competition est utilisée pour jouer les stratégies entre elles.

In [6]:
class Competition(object):
    # Takes two strategies (one for each player), play them against each other once and declare an outcome

    def play(self, player1, player2, verbose=False):
        state = State()
        count_opt = 0
        play_length = 0
        if verbose:
            print("\nMatch between Player %s (as Player 1) and Player %s (as Player 2)"
                  % (player1.name, player2.name))

        while state.compute_outcome() == -1:
            if verbose:
                state.print_state()
            if state.player == 1:
                i, j = player1.play(state, verbose)
                i=1
                j=1
                if verbose:
                    print("Player 1 chooses (",i,",",j,")")
                state.update_state(i, j)
            else:
                i, j = player2.play(state, verbose)
                if verbose:
                    print("Player 2 chooses (%d,%d)" % (i, j))
                state.update_state(i, j)

        if verbose:
            print("Final state")
            state.print_state()
            if state.outcome == 1:
                print("Player 1 won")
            elif state.outcome == 0:
                print("Player 2 won")
            else:
                print("It's a tie!")
        return state.outcome,count_opt,play_length

    # Takes two strategies (one for each player) and play them against each other for a number of games
    def compete(self, player1, player2, games = 500):
        player1_win = 0.0
        player2_win = 0.0
        count_ties = 0.0
        count_opt_tot = 0
        count_length_tot = 0
        for cpt in range(games):
            print ("Play N°",cpt,":ended")
            outcome,count_opt,play_length = \
                self.play(player1,player2, verbose = False)
            count_opt_tot += count_opt
            count_length_tot += play_length
            if outcome == 1:
                player1_win += 1
            elif outcome == 0:
                player2_win += 1  
            elif outcome == 0.5:
                count_ties += 1
                
        print("\nCompetition of Player %s (as Player 1) against Player %s (as Player 2):"
              "\n %d plays, Player 1 wins %.02f, Player 2 wins %.02f (Ties %.02f)"
              % (player1.name, player2.name, games, player1_win / games, player2_win / games, count_ties / games))

    

  * Pour lancer un joueur "random" :

player_random1 = Player(name='random1', strategy='random')
  * Pour lancer un joueur "humain" :

player_human1 = Player(name='human1', strategy='human')

  * Pour instancier lancer une compétition :

competition = Competition()
competition.play(player_random,player_random,verbose=True)

## Faites quelques tests



In [7]:
print(' ')

Ecrivez les fonctions max et min de la classe Player afin d'implémenter l'algorithme MiniMax :

In [8]:
class Player():
    def __init__(self, name = 'Anonymous', strategy ='human'):
        self.name = name
        self.strategy = strategy

    def max(self,state):
        maxv = -2 

        px = None
        py = None

        return (maxv, px, py)


    def min(self,state):
        minv = +2 

        qx = None
        qy = None
        return (minv, qx, qy)

    def play(self, state, verbose = False):      
        if self.strategy == 'random':  
            if (verbose):
                print("Joueur ",state.player,": Je joue au hasard.")
            return random.choice(state.legal_plays())
        elif self.strategy == 'minmax':  
            if (verbose):
                print("Joueur ",state.player,": Je joue MinMax.")
            (m, px, py) = self.max(state)   
            return (px, py)
        elif self.strategy == 'human':
            if (verbose):
                print("Joueur ",state.player,": Un humain joue...")
            while True:   
                px = int(input('Insert the X coordinate: '))
                py = int(input('Insert the Y coordinate: '))
                if (px, py) in state.legal_plays():
                    return (px, py)
                else:
                    print('The move is not valid! Try again.')
        else:
            print("Je ne connais pas cette stratégie : je joue au hasard !")
            return random.choice(state.legal_plays())

## Lancez des compétitions avec les joueurs suivants :

  * player_minmax = Player(name='minmax', strategy='minmax') 

  * player_random = Player(name='random', strategy='random')

  * player_human1 = Player(name='human1', strategy='human')


In [9]:
print()



Rajoutons maintenant alphabeta :

In [10]:
class Player():
    def __init__(self, name = 'Anonymous', strategy ='human'):
        self.name = name
        self.strategy = strategy

    def max(self,state):
        maxv = -2 
        px = None
        py = None

        return (maxv, px, py)


    def min(self,state):
        minv = +2 
        qx = None
        qy = None
        return (minv, qx, qy)


    def max_alpha_beta(self, state, alpha, beta):
        maxv = -2 
        px = None
        py = None
        return (maxv, px, py)

    def min_alpha_beta(self, state, alpha, beta):
        minv = +2 
        qx = None
        qy = None
        return (minv, qx, qy)
 
    def play(self, state, verbose = False):      
        if self.strategy == 'random':  
            if (verbose):
                print("Joueur ",state.player,": Je joue au hasard.")
            return random.choice(state.legal_plays())
        elif self.strategy == 'minmax':  
            if (verbose):
                print("Joueur ",state.player,": Je joue MinMax.")
            (m, px, py) = self.max(state)    
            return (px, py)
        elif self.strategy == 'alphabeta':  
            if (verbose):
                print("Joueur ",state.player,": Je joue AlphaBeta.")
            (m, px, py) = self.max_alpha_beta(state,-2,2)  
            print("Je joue :",px,py)
            return (px, py)
        elif self.strategy == 'human':
            if (verbose):
                print("Joueur ",state.player,": Un humain joue...")
            while True:   
                px = int(input('Insert the X coordinate: '))
                py = int(input('Insert the Y coordinate: '))
                if (px, py) in state.legal_plays():
                    return (px, py)
                else:
                    print('The move is not valid! Try again.')
        else:
            print("Je ne connais pas cette stratégie : je joue au hasard !")
            return random.choice(state.legal_plays())

## Expérimentez...

In [11]:
player_minmax = Player(name='minmax', strategy='minmax') 
player_random = Player(name='random', strategy='random')
player_alphabeta1 = Player(name='alphabeta', strategy='alphabeta')
player_alphabeta2 = Player(name='alphabeta', strategy='alphabeta')
player_human1 = Player(name='human1', strategy='human')
player_human2 = Player(name='human2', strategy='human')

competition = Competition()

competition.play(player_alphabeta1,player_random,verbose=True)



Match between Player alphabeta (as Player 1) and Player random (as Player 2)
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
Joueur  1 : Je joue AlphaBeta.
Je joue : None None
Player 1 chooses ( 1 , 1 )
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
Joueur  2 : Je joue au hasard.
Player 2 chooses (0,0)
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
Joueur  1 : Je joue AlphaBeta.
Je joue : None None
Player 1 chooses ( 1 , 1 )
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
Joueur  2 : Je joue au hasard.
Player 2 chooses (2,1)
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|   | o |   | 
-------------
Joueur  1 : Je joue AlphaBeta.
Je joue : None None
Player 1 chooses ( 1 , 1 )
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|  

(0, 0, 0)